In [8]:
!pip install fairing
!pip uninstall -y fairing
!pip install numpy==1.15.0 kubernetes==6.0.0
!pip install --index-url https://test.pypi.org/simple/ fairing

polyaxon-schemas 0.1.6 has requirement marshmallow==2.15.0, but you'll have marshmallow 3.0.0b12 which is incompatible.
polyaxon-schemas 0.1.6 has requirement numpy==1.14.2, but you'll have numpy 1.15.0 which is incompatible.
polyaxon-schemas 0.1.6 has requirement python-dateutil==2.7.0, but you'll have python-dateutil 2.7.3 which is incompatible.
polyaxon-schemas 0.1.6 has requirement PyYAML==3.13, but you'll have pyyaml 3.12 which is incompatible.
polyaxon-helper 0.1.4 has requirement requests==2.18.4, but you'll have requests 2.19.1 which is incompatible.
polyaxon-client 0.1.6 has requirement requests==2.18.4, but you'll have requests 2.19.1 which is incompatible.
metaml 0.1 has requirement docker==2.7.0, but you'll have docker 3.4.1 which is incompatible.
phillytools 3.0.1.dev0 has requirement docker==3.2.1, but you'll have docker 3.4.1 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --up

polyaxon-schemas 0.1.6 has requirement marshmallow==2.15.0, but you'll have marshmallow 3.0.0b12 which is incompatible.
polyaxon-schemas 0.1.6 has requirement numpy==1.14.2, but you'll have numpy 1.15.0 which is incompatible.
polyaxon-schemas 0.1.6 has requirement python-dateutil==2.7.0, but you'll have python-dateutil 2.7.3 which is incompatible.
polyaxon-schemas 0.1.6 has requirement PyYAML==3.13, but you'll have pyyaml 3.12 which is incompatible.
polyaxon-helper 0.1.4 has requirement requests==2.18.4, but you'll have requests 2.19.1 which is incompatible.
polyaxon-client 0.1.6 has requirement requests==2.18.4, but you'll have requests 2.19.1 which is incompatible.
metaml 0.1 has requirement docker==2.7.0, but you'll have docker 3.4.1 which is incompatible.
phillytools 3.0.1.dev0 has requirement docker==3.2.1, but you'll have docker 3.4.1 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --up

In [9]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Trains and Evaluates the MNIST network using a feed dictionary."""
import argparse
import os
import sys
import time
import random
import logging

from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist

from fairing.train import Train
from fairing.architectures.kubeflow.basic import BasicArchitecture

In [10]:
INPUT_DATA_DIR = '/tmp/tensorflow/mnist/input_data/'
MAX_STEPS = 2000
BATCH_SIZE = 100
LEARNING_RATE = 0.3
HIDDEN_1 = 128
HIDDEN_2 = 32

# HACK: Ideally we would want to have a unique subpath for each instance of the job, but since we can't
# we are instead appending HOSTNAME to the logdir
LOG_DIR = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                       'tensorflow/mnist/logs/fully_connected_feed/', os.getenv('HOSTNAME', ''))
MODEL_DIR = os.path.join(LOG_DIR, 'model.ckpt')

In [11]:

@Train(package={'repository': 'wbuchwalter'}, builder='knative',
       architecture=BasicArchitecture())
class MyModel(object):
    def train(self):
        self.data_sets = input_data.read_data_sets(INPUT_DATA_DIR)
        self.images_placeholder = tf.placeholder(
            tf.float32, shape=(BATCH_SIZE, mnist.IMAGE_PIXELS))
        self.labels_placeholder = tf.placeholder(tf.int32, shape=(BATCH_SIZE))

        logits = mnist.inference(self.images_placeholder,
                                 HIDDEN_1,
                                 HIDDEN_2)

        self.loss = mnist.loss(logits, self.labels_placeholder)
        self.train_op = mnist.training(self.loss, LEARNING_RATE)
        self.summary = tf.summary.merge_all()
        init = tf.global_variables_initializer()
        saver = tf.train.Saver()
        self.sess = tf.Session()
        self.summary_writer = tf.summary.FileWriter(LOG_DIR, self.sess.graph)
        self.sess.run(init)

        data_set = self.data_sets.train
        for step in xrange(MAX_STEPS):
            images_feed, labels_feed = data_set.next_batch(BATCH_SIZE, False)
            feed_dict = {
                self.images_placeholder: images_feed,
                self.labels_placeholder: labels_feed,
            }

            _, loss_value = self.sess.run([self.train_op, self.loss],
                                     feed_dict=feed_dict)
            if step % 100 == 0:
                print("At step {}, loss = {}".format(step, loss_value))
                summary_str = self.sess.run(self.summary, feed_dict=feed_dict)
                self.summary_writer.add_summary(summary_str, step)
                self.summary_writer.flush()

In [12]:
model = MyModel()
model.train()


Building docker image wbuchwalter/fairing-build:b1c20aff...
Build finished successfully.
Training(s) launched.


SystemExit: 0

/home/will/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
